In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-yo6wapgx
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-yo6wapgx
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [26]:
%%cu
#include <cuda_runtime.h>
#include <iostream>
#include <fstream>
#include <cstring>
#include <vector>

// Define Student structure
struct Student {
    char name[256];
    int credits;
    double gpa;
};

// Define Result structure
struct Result {
    char data[256];
};

__device__ int gradeFromScore(double gpa) {

    if (gpa >= 8.5) return 4;
    else if (gpa >= 7.5) return 3;
    else if (gpa >= 6.0) return 2;
    else if (gpa >= 5.0) return 1;
    else return 0;
}

__global__ void kernel(Student* students, Result* results, int numStudents, int* resultCounter) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < numStudents) {
        Student s = students[idx];
        if (s.name[0] > 'K') {
            // Convert name to uppercase
            for (int i = 0; s.name[i] != '\0'; i++) {
                if (s.name[i] >= 'a' && s.name[i] <= 'z') {
                    s.name[i] = s.name[i] - 'a' + 'A';
                }
            }

            // Prepare result string
            char resultString[256];
            int cursor = 0;
            for (int i = 0; s.name[i] != '\0'; i++) {
                resultString[cursor++] = s.name[i];
            }
            resultString[cursor++] = '-';

            // Convert GPA to numeric grade
            int grade = gradeFromScore(s.gpa);
            // Append grade as a string
            resultString[cursor++] = '0' + grade;

            // Check credit conditions
            if (s.credits >= 200) {
                // Satisfied or over-satisfied
                if (s.credits > 200) {
                    resultString[cursor++] = ' ';


                    resultString[cursor++] = 'O';
                      resultString[cursor++] = '/';

                }


                resultString[cursor++] = 'S';
            }


            resultString[cursor] = '\0';
            __syncthreads();

            // Write to result array using atomic operation
            int pos = atomicAdd(resultCounter, 1);
            for (int i = 0; resultString[i] != '\0'; i++) {
                results[pos].data[i] = resultString[i];
            }
            results[pos].data[cursor] = '\0';
        }
    }
}

std::vector<Student> readStudentsFromFile(const char* filename) {
    std::vector<Student> students;
    std::ifstream file(filename);
    std::string line;

    while (std::getline(file, line)) {
        Student s;
        sscanf(line.c_str(), "%[^,],%d,%lf", s.name, &s.credits, &s.gpa);
        students.push_back(s);
    }

    file.close();
    return students;
}

int main() {
    // Read students from file
    std::vector<Student> studentVector = readStudentsFromFile("data.txt");
    int numStudents = studentVector.size();

    // Prepare arrays for CUDA
    Student* h_students = new Student[numStudents];
    Result* h_results = new Result[numStudents];
    int h_resultCounter = 0;

    for (int i = 0; i < numStudents; ++i) {
        h_students[i] = studentVector[i];
    }

    // Allocate memory on GPU
    Student* d_students;
    Result* d_results;
    int* d_resultCounter;
    cudaMalloc(&d_students, numStudents * sizeof(Student));
    cudaMalloc(&d_results, numStudents * sizeof(Result));
    cudaMalloc(&d_resultCounter, sizeof(int));

    // Copy data from host to device
    cudaMemcpy(d_students, h_students, numStudents * sizeof(Student), cudaMemcpyHostToDevice);
    cudaMemcpy(d_resultCounter, &h_resultCounter, sizeof(int), cudaMemcpyHostToDevice);

    // Launch kernel
    int blockSize = 32; // or another appropriate value
   int numBlocks = (numStudents + blockSize - 1) / blockSize;

    kernel<<<numBlocks, blockSize>>>(d_students, d_results, numStudents, d_resultCounter);

    // Copy results back to host
    cudaMemcpy(h_results, d_results, numStudents * sizeof(Result), cudaMemcpyDeviceToHost);
    cudaMemcpy(&h_resultCounter, d_resultCounter, sizeof(int), cudaMemcpyDeviceToHost);

    // Write results to a file
    std::ofstream outFile("result.txt");
    for (int i = 0; i < h_resultCounter; i++) {
        outFile << h_results[i].data << std::endl;
    }
    outFile.close();


    cudaFree(d_students);
    cudaFree(d_results);
    cudaFree(d_resultCounter);

    return 0;
}
